# <font color='#11a642' size='6'> **Импорт и установка библиотек**

In [ ]:
!pip install category_encoders -q

In [ ]:
!pip install shap -q

In [ ]:
#from google.colab import drive

In [ ]:
import zipfile
import json
import random
import pandas as pd
pd.set_option("display.max_columns", 100)
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder



from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree

from sklearn.impute import KNNImputer, SimpleImputer

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# import category_encoders as ce
from category_encoders.basen import BaseNEncoder
from category_encoders import TargetEncoder

from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve, f1_score

from sklearn.inspection import permutation_importance
import shap

import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
from category_encoders import TargetEncoder

In [ ]:
def calculate_metrics_and_plot_roc_comparison(
    model,
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_test: np.ndarray,
    y_test: np.ndarray,
    best_threshold: float
) -> dict:
    """
    Рассчитывает метрики классификации и строит ROC-кривые для train и test выборок на одном графике.

    Параметры:
    ----------
    model :
        Обученная модель, реализующая метод predict_proba.

    X_train : np.ndarray
        Признаки обучающей выборки.

    y_train : np.ndarray
        Истинные метки классов обучающей выборки.

    X_test : np.ndarray
        Признаки тестовой выборки.

    y_test : np.ndarray
        Истинные метки классов тестовой выборки.

    best_threshold: float
      Порог для бинарного прогноза.

    Возвращает:
    -----------
    metrics : dict
        Словарь с метриками для обеих выборок.
    """
    results = {}

    for X, y, sample_type in [(X_train, y_train, "train"), (X_test, y_test, "test")]:
        # Предсказание вероятностей
        y_proba = model.predict_proba(X)[:, 1]

        # Прогноз с учетом порога
        y_pred = (y_proba >= 0.5).astype(int)

        # Вычисление метрик
        precision = precision_score(y, y_pred)
        recall = recall_score(y, y_pred)
        roc_auc = roc_auc_score(y, y_proba)
        fpr, tpr, _ = roc_curve(y, y_proba)

        # Сохранение результатов
        results[sample_type] = {
            'precision': precision,
            'recall': recall,
            'roc_auc': roc_auc,
            'fpr': fpr,
            'tpr': tpr,
            'threshold': best_threshold
        }


    # Построение ROC-кривых на одном графике
    plt.figure(figsize=(10, 8))

    colors = {'train': 'blue', 'test': 'red'}
    for sample_type in ['train', 'test']:
        plt.plot(
            results[sample_type]['fpr'],
            results[sample_type]['tpr'],
            color=colors[sample_type],
            lw=2,
            label=f'{sample_type.capitalize()} (AUC = {results[sample_type]["roc_auc"]:.2f})'
        )

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title(f'ROC Curves Comparison ({type(model).__name__})', fontsize=14)
    plt.legend(loc="lower right", fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.show()

    return results

# <font color='#11a642' size='6'> **Загрузка данных**

In [ ]:
def load_dataset(from_kaggle:bool = False) -> pd.DataFrame:
  '''
  Функция скачивает данные с сайта kaggle, если установлен from_kaggle=True,
  инчае архив считывается по ссылке с гугл диска (такой способ удобен тем, у кого нет доступа к kaggle)
  params:
      - from_kaggle - индикатор откуда скачивать данные (True - c сайта kaggle, False -  c google диска)
  return:
      - pd.DataFrame

  '''
  '''
  if from_kaggle:
    # запросит разрешение к гугл диску, необходимо дать это разрешение
    drive.mount('/content/drive')
    # установим kaggle
    !pip install kaggle -q
    !mkdir ~/.kaggle
    # копируем kaggle.json (предварительно, необходимо сгенерить токен на
    # сайте kaggle и сохранить к себе на гугл диск) в папку ~/.kaggle/
    !cp "/content/drive/MyDrive/Colab Notebooks/config/kaggle.json" ~/.kaggle/
    !kaggle competitions download -c playground-series-s5e7
    # !kaggle competitions download -c playground-series-s5e3
  else:
    !gdown 1-730JF1IWA5e_ejuXWLmkkzHFvudisdp
  '''
  # распаковка архива
  zip_ref = zipfile.ZipFile('playground-series-s5e7.zip', 'r')
  zip_ref.extractall()
  zip_ref.close()
  df_train = pd.read_csv('train.csv')
  df_test = pd.read_csv('test.csv')
  df_sample_submission = pd.read_csv('sample_submission.csv')
  return df_train, df_test, df_sample_submission

Time_spent_Alone: Количество часов, проводимых в одиночестве ежедневно (0–11).

Stage_fear: Наличие страха сцены (Да/Нет).

Social_event_attendance: Частота посещения социальных мероприятий (0–10).

Going_outside: Частота выхода из дома (0–7).

Drained_after_socializing: Чувство опустошения после общения (Да/Нет).

Friends_circle_size: Количество близких друзей (0–15).

Post_frequency: Частота публикаций в соцсетях (0–10).

Personality: Целевая переменная (Экстраверт/Интроверт).

In [ ]:
df_train, df_test, df_sample_submission = load_dataset(from_kaggle = False)

In [ ]:
df_train

In [ ]:
df_train.columns = (col.lower().replace("(", "_").replace(")", "").replace(" ", "_") for col in df_train.columns)


In [ ]:
df_train.head()

In [ ]:
df_test.columns = (col.lower().replace("(", "_").replace(")", "").replace(" ", "_") for col in df_test.columns)


In [ ]:
df_train['personality'].unique()

In [ ]:
dict_personality = {'Extrovert':0, 'Introvert':1}
df_train['int_personality'] = df_train['personality'].map(dict_personality)

In [ ]:
df_train.shape

Time_spent_Alone: Количество часов, проводимых в одиночестве ежедневно (0–11).

Stage_fear: Наличие страха сцены (Да/Нет).

Social_event_attendance: Частота посещения социальных мероприятий (0–10).

Going_outside: Частота выхода из дома (0–7).

Drained_after_socializing: Чувство опустошения после общения (Да/Нет).

Friends_circle_size: Количество близких друзей (0–15).

Post_frequency: Частота публикаций в соцсетях (0–10).

Personality: Целевая переменная (Экстраверт/Интроверт).

In [ ]:
df_train.describe(include='all').T

In [ ]:
df_train['time_spent_alone'].value_counts(normalize=True, dropna=False)

In [ ]:
df_train['drained_after_socializing'].value_counts(normalize=True, dropna=False)

In [ ]:
df_train['stage_fear'].value_counts(normalize=True, dropna=False)

In [ ]:
df_train['drained_after_socializing'].value_counts(normalize=True, dropna=False)

In [ ]:
df_train['social_event_attendance'].value_counts(normalize=True, dropna=False)

> social_event_attendance -> 0

In [ ]:
df_train['friends_circle_size'].isna().mean()

> friends_circle_size -> mean

> точка улучшения: можно заменить моделью




In [ ]:
df_train.groupby('personality')['friends_circle_size'].mean()

In [ ]:
df_train['post_frequency'].isna().mean()

In [ ]:
df_train[pd.isna(df_train['post_frequency'])].isna().sum()


In [ ]:
df_train.isna().sum()

In [ ]:
import missingno as msno
msno.matrix(df_train)


In [ ]:
#df_train = df.copy()

## <font color='#11a642' size='5'> Определить роли для независимых признаков

In [ ]:
id_features = ['id', ]
target = 'int_personality'
exclude_features = ['personality',]

num_features = [col for col in df_train.select_dtypes(include='number').columns
                if col not in id_features + [target, ] + exclude_features]

cat_features = [col for col in df_train.select_dtypes(exclude='number').columns
                if col not in id_features + [target, ] + exclude_features]


In [ ]:
num_features

In [ ]:
cat_features

# <font color='#11a642' size='6'> **Разведочный анализ данных**

In [ ]:
df_train.info()

In [ ]:
df_train.isna().sum()[df_train.isna().sum()>0]

In [ ]:
df_train

In [ ]:
sns.barplot(df_train[target].value_counts(normalize=True))

In [ ]:
label_encoder = LabelEncoder()
df_encoded = df_train.copy()
df_encoded['stage_fear'] = label_encoder.fit_transform(df_encoded['stage_fear'])
df_encoded['drained_after_socializing'] = label_encoder.fit_transform(df_encoded['drained_after_socializing'])
df_encoded['personality'] = label_encoder.fit_transform(df_encoded['personality'])

plt.figure(figsize=(12, 8))
correlation_matrix = df_encoded.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Корреляционная матрица')
plt.show()

# <font color='#11a642' size='6'> **Разделите данные на трейн и тест**

In [ ]:
df_train.head()

In [ ]:
df_train[num_features + cat_features ].loc[75, :]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train[num_features + cat_features ],
                                                    df_train[target],
                                                    stratify=df_train[target],
                                                    train_size=0.75,
                                                    random_state=12345)

In [ ]:
X_test.head()

In [ ]:
X_test.shape , X_train.shape

# <font color='#11a642' size='6'> **Подготовка данных к обучению**

## c pipeline

In [ ]:
# SimpleImputer + OHE
cat_pipe = Pipeline(
    [

        (
            'enc',
            TargetEncoder(cols = cat_features)

        ),
        (
            'imputer',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),

    ]
)

In [ ]:
# TODO разделить pipeline: некоторые переменные -0 , некоторые среднее (или модель)
# SimpleImputer + StandardScaler
num_pipe = Pipeline(
    [
        (
            'scaler',
            StandardScaler()
        ),
        (
            'imputer',
            SimpleImputer(missing_values=np.nan, strategy='mean')
        ),

    ]
)

In [ ]:
data_preprocessor = ColumnTransformer(
    [
        ('cat', cat_pipe, cat_features),
        ('num', num_pipe, num_features),
    ],
    remainder='passthrough'
)

In [ ]:
type(data_preprocessor.fit_transform(X_train, y_train))

In [ ]:
data_preprocessor.get_feature_names_out()

In [ ]:
X_train_p = pd.DataFrame(
    data_preprocessor.fit_transform(X_train, y_train),
    columns=data_preprocessor.get_feature_names_out()
)

X_test_p = pd.DataFrame(
    data_preprocessor.transform(X_test),
    columns=data_preprocessor.get_feature_names_out()
)

In [ ]:
X_train_p.head()

In [ ]:
X_test_p.head()

# <font color='#11a642' size='6'> **Построение моделей**

## <font color='blue' size='5'> **Логистическая регрессия**

In [ ]:
model_log_reg = LogisticRegression()

In [ ]:
model_log_reg.fit(X_train_p, y_train)

In [ ]:
y_pred_test = model_log_reg.predict_proba(X_test_p)

In [ ]:
y_pred_train = model_log_reg.predict_proba(X_train_p)

In [ ]:
X_train_p.columns

In [ ]:
model_log_reg.coef_[0]

In [ ]:
feature_imp = pd.DataFrame({'feature': list(X_train_p.columns), 'coef': model_log_reg.coef_[0]} )
feature_imp.sort_values('coef', ascending=False)

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(feature_imp.sort_values('coef', ascending=False), y = 'feature', x='coef')

In [ ]:
res = calculate_metrics_and_plot_roc_comparison(
    model_log_reg,
    X_train_p,
    y_train,
    X_test_p,
    y_test,
    0.5)

## <font color='blue' size='5'> **Дерево решений**

In [ ]:
params = {
    'criterion': ['gini',],
    'max_depth': [5, 10, 15,], # range(3, 15, 3)
    'min_samples_leaf': [10, 15, 20],
    'ccp_alpha': np.arange(0.1, 2, 0.5)
}

In [ ]:
tree_classifier = DecisionTreeClassifier(random_state=42)

In [ ]:
# Создаем объект GridSearchCV
grid_search = GridSearchCV(estimator=tree_classifier, param_grid=params, cv=5, n_jobs=-1, scoring='accuracy')


In [ ]:
grid_search.fit(X_train_p, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
best_tree = grid_search.best_estimator_

In [ ]:
calculate_metrics_and_plot_roc_comparison(
    model_log_reg,
    X_train_p,
    y_train,
    X_test_p,
    y_test,
    0.5)

In [ ]:
pred_train = best_tree.predict_proba(X_train_p)
pred_test = best_tree.predict_proba(X_test_p)

In [ ]:
pd.DataFrame(
    {"feature": X_train_p.columns, "importance": best_tree.feature_importances_}
).sort_values(by="importance", ascending=False).reset_index(drop=True)

In [ ]:
# второй способ отрисовать деревья с помощью plot_tree
plt.figure(figsize=(10, 10))
plot_tree(best_tree, feature_names=X_train_p.columns, filled=True, rounded=True,  fontsize=10)
plt.show()
plt.savefig('tree.png')

## <font color='blue'> **Random Forest**

In [ ]:
# итоговый пайплайн: подготовка данных и модель RandomForestClassifier
rf_cl = Pipeline(
   [
       ('preprocessor', data_preprocessor),
       ('rf_models', RandomForestClassifier(random_state = 47))

   ]
)

In [ ]:
rf_cl.fit(X_train, y_train)

In [ ]:
pred_train = rf_cl.predict_proba(X_train)

In [ ]:
pred_test = rf_cl.predict_proba(X_test)

In [ ]:
rf_cl.steps

In [ ]:
from scipy.stats import randint

param_distributions = {
    'rf_models__n_estimators': randint(50, 400),
    'rf_models__criterion': ['gini'],
    'rf_models__max_depth': randint(3, 15),
    'rf_models__min_samples_split': randint(2, 15),
    'rf_models__min_samples_leaf': randint(1, 15),
    'rf_models__max_features': ['log2', 'sqrt'],
}

In [ ]:
random_search = RandomizedSearchCV(
    estimator=rf_cl,
    param_distributions=param_distributions,
    n_iter=30,            # Количество случайных комбинаций (можешь увеличить)
    cv=5,                 # 5-fold CV
    scoring='f1',         # Оценка по F1
    verbose=2,
    n_jobs=-1,            # Используем все ядра
    random_state=42
)

random_search.fit(X_train, y_train)

print("Лучшие параметры:", random_search.best_params_)
print("Лучший F1 на кросс-валидации:", random_search.best_score_)

In [ ]:
'''param_grid = {
    'rf_models__n_estimators': [100, 300],  # Количество деревьев в лесу
    'rf_models__criterion': ['gini', ],      # Функция для измерения качества разделения: индекс Джини или энтропия
    'rf_models__max_depth': [5, 7, 10],        # Максимальная глубина дерева: ограничение глубины для избежания переобучения
    'rf_models__min_samples_split': [5, 10],       # Минимальное количество выборок, необходимых для разделения внутреннего узла
    'rf_models__min_samples_leaf': [7, 10],         # Минимальное количество выборок, необходимых для узла листа
    'rf_models__max_features': ['log2'], # Количество признаков для поиска наилучшего разделения
                                                       # 'sqrt' - корень из общего кол-ва признаков, 'log2' - логарифм по основанию 2 от общего кол-ва признаков, None - все признаки
}'''

In [ ]:
grid_search = GridSearchCV(rf_cl, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2)


In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
# Получаем лучшее значение параметров, найденных в процессе кросс-валидации
print("Лучшие параметры:", grid_search.best_params_)

In [ ]:
pred_train = grid_search.best_estimator_.predict_proba(X_train)

In [ ]:
best_rf_model = grid_search.best_estimator_

In [ ]:
pred_test = best_rf_model.predict_proba(X_test)

In [ ]:
res_rf = calculate_metrics_and_plot_roc_comparison(
    best_rf_model,
    X_train,
    y_train,
    X_test,
    y_test,
    0.5)

# <font color='blue'> **Интерпретация лучшей модели**

In [ ]:
best_rf_model.steps[1][1]

In [ ]:
%%time
explainer_tree = shap.Explainer(best_rf_model.steps[1][1].predict, X_test_p)
shap_values_tree = explainer_tree(X_test_p)

In [ ]:
i = 1
shap.plots.waterfall(shap_values_tree[i], max_display=14)

In [ ]:
shap.plots.beeswarm(shap_values_tree)

# <font color='blue'> **Сохранение модели**

In [ ]:
import pickle

# Сохранение пайплайна в файл
with open('pipeline_best_model.pkl', 'wb') as f:
    pickle.dump(best_rf_model, f)

print("Pipeline сохранен в файл 'pipeline_best_model.pkl'.")


# <font color='blue'> **Инференс модели**

In [ ]:
# Загрузка пайплайна из файла
with open('pipeline_best_model.pkl', 'rb') as f:
    loaded_pipeline = pickle.load(f)

print("Pipeline загружен.")


In [ ]:
loaded_pipeline.steps[1][1]
# .get_feature_names_out()

In [ ]:
df_test['pred'] = loaded_pipeline.predict(df_test)

In [ ]:
df_test

In [ ]:
#dict_personality = {'Extrovert':0, 'Introvert':1}
dict_personality_rev = {0:'Extrovert', 1:'Introvert'}

In [ ]:
df_test['Personality'] = df_test['pred'].map(dict_personality_rev)

In [ ]:
df_test

In [ ]:
df_test[['id', 'Personality']].to_csv('submission_.csv', index=False)

In [ ]:
!kaggle competitions submit -c playground-series-s5e7 -f submission.csv -m "1 commit"